In [11]:
import os
import numpy as np

import torch
import torch.nn as nn 
import torch.nn.functional as F 
from collections import OrderedDict
from PIL import Image
import torchvision.transforms as transforms

from models.iresnet import iresnet50
from models.evidential import edl_mse_loss, edl_digamma_loss, edl_log_loss, relu_evidence, exp_evidence, get_device, one_hot_embedding

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm

In [12]:
num_classes = 8631
pre_trained = "ckpt/ArcFace-8631.pth"
device = "cuda:0"

In [13]:
model = iresnet50(num_classes)
if pre_trained is not None and os.path.exists(pre_trained):
    # No related
    model_dict = model.state_dict()
    pretrained_param = torch.load(pre_trained)
    try:
        pretrained_param = pretrained_param.state_dict()
    except:
        pass

    new_state_dict = OrderedDict()
    for k, v in pretrained_param.items():
        if k in model_dict:
            new_state_dict[k] = v
            print("Load parameter {}".format(k))
        elif k[7:] in model_dict:
            new_state_dict[k[7:]] = v
            print("Load parameter {}".format(k[7:]))
        # else:
        #     print("Parameter {} unload.".format(k[7:]))

    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict)
    print("Success load pre-trained model {}".format(pre_trained))
else:
    print("not load pretrained")

# Set to device
model.to(device)
model.eval()

Load parameter weight
Load parameter conv1.weight
Load parameter bn1.weight
Load parameter bn1.bias
Load parameter bn1.running_mean
Load parameter bn1.running_var
Load parameter bn1.num_batches_tracked
Load parameter prelu.weight
Load parameter layer1.0.bn1.weight
Load parameter layer1.0.bn1.bias
Load parameter layer1.0.bn1.running_mean
Load parameter layer1.0.bn1.running_var
Load parameter layer1.0.bn1.num_batches_tracked
Load parameter layer1.0.conv1.weight
Load parameter layer1.0.bn2.weight
Load parameter layer1.0.bn2.bias
Load parameter layer1.0.bn2.running_mean
Load parameter layer1.0.bn2.running_var
Load parameter layer1.0.bn2.num_batches_tracked
Load parameter layer1.0.prelu.weight
Load parameter layer1.0.conv2.weight
Load parameter layer1.0.bn3.weight
Load parameter layer1.0.bn3.bias
Load parameter layer1.0.bn3.running_mean
Load parameter layer1.0.bn3.running_var
Load parameter layer1.0.bn3.num_batches_tracked
Load parameter layer1.0.downsample.0.weight
Load parameter layer1.0.

IResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=64)
  (layer1): Sequential(
    (0): IBasicBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): IBas

In [14]:
transforms = transforms.Compose([
            transforms.Resize((112,112)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])

In [15]:
img = Image.open(img_path)
img = transforms(img).unsqueeze(0)

In [19]:
model(img.to(device))

tensor([[ 0.8906, -0.0348,  0.0472,  ...,  0.0828,  0.0342,  0.0935]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [20]:
batch_size = 1  #批处理大小
input_shape = (3, 112, 112)   #输入数据,改成自己的输入shape

In [21]:
x = torch.randn(batch_size, *input_shape)   # 生成张量
x = x.to(device)
export_onnx_file = "ArcFace-R101-8631.onnx"		# 目的ONNX文件名
torch.onnx.export(model,
                    x,
                    export_onnx_file,
                    opset_version=10,
                    do_constant_folding=True,	# 是否执行常量折叠优化
                    input_names=["input"],	# 输入名
                    output_names=["output"],	# 输出名
                    dynamic_axes={"input":{0:"batch_size"},  # 批处理变量
                                    "output":{0:"batch_size"}})

## ONNX

In [1]:
import onnx_tf.backend
import onnx

import numpy as np
from PIL import Image
import torchvision.transforms as transforms

2023-04-18 16:02:09.124217: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 16:02:09.499293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 16:02:10.611497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode

In [2]:
transforms = transforms.Compose([
            transforms.Resize((112,112)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])

In [3]:
img_path = "image/0001_01.jpg"
model_path = "ckpt/ArcFace-R50-8631.onnx"

In [4]:
img = Image.open(img_path)
img = np.array([transforms(img).numpy()])

In [5]:
onnx_model = onnx.load(model_path)
tf_model = onnx_tf.backend.prepare(onnx_model, device='GPU:0')

2023-04-18 16:02:32.070273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21733 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [6]:
result = tf_model.run(img)

2023-04-18 16:02:46.228742: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.3.2 but source was compiled with: 8.6.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2023-04-18 16:02:46.230075: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops.cc:1068 : UNIMPLEMENTED: DNN library is not found.
2023-04-18 16:02:46.230156: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): UNIMPLEMENTED: DNN library is not found.
	 [[{{node convolution}}]]


UnimplementedError: Graph execution error:

Detected at node 'convolution' defined at (most recent call last):
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/cry/anaconda3/envs/hsic/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/cry/anaconda3/envs/h

: 

In [39]:
prob = np.argsort(result[0])[::-1]

In [7]:
result[0]

array([[ 0.8906132 , -0.03505185,  0.04598166, ...,  0.08282372,
         0.03448495,  0.09236697]], dtype=float32)